################################################################################################################
# CLASSIFYING 911 CALLS #
################################################################################################################

## DESCRIPTION :: Emergency (911) Calls: Fire, Traffic, EMS for Montgomery County, PA

- You can get a quick introduction to this Dataset with this kernel: Dataset Walk-through @ https://www.kaggle.com/mchirico/d/mchirico/montcoalert/dataset-walk-through-911

## SOURCE :: KAGGLE 
* https://www.kaggle.com/mchirico/montcoalert

## WHY I'M INTERESTED / MY DOMAIN EXPERTISE

I've worked as an EMT and completed paramedic school in 2012.  Two things surprised me from my paramedic internship:
- we spent a lot of time driving from post to post, filling the space left when another unit went on a call
- we responded to a lot of calls that did not need paramedic-level care (needed basic - not advanced - life support based on clinical criteria)
- my preceptor once remarked that she got most calls for dead on arrival in the morning...when someone woke up next to their dead spouse

## WHAT I'M HOPING TO GAIN FROM THIS
- find hotspots & trends for
- - all calls
- - specific types of calls
- - time
- - level of response requested

## HAVE TO DISCUSS WHAT I'M OPTIMIZING FOR AND WHY BASED ON THE DOMAIN:
- accuracy
- sensitivity
- specificity
- precision
- rocauc // threshold-ambiguous

> # BASIC PACKAGES IMPORTS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

> # DATA IMPORT

In [4]:
data = pd.read_csv('http://localhost:8888/files/DATA/911.csv')
data.shape

(120518, 9)

### overview

In [51]:
data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120518 entries, 0 to 120517
Data columns (total 9 columns):
lat          120518 non-null float64
lng          120518 non-null float64
desc         120518 non-null object
zip          104887 non-null float64
title        120518 non-null object
timeStamp    120518 non-null object
twp          120467 non-null object
addr         119910 non-null object
e            120518 non-null int64
dtypes: float64(3), int64(1), object(5)
memory usage: 8.3+ MB


lat              0
lng              0
desc             0
zip          15631
title            0
timeStamp        0
twp             51
addr           608
e                0
dtype: int64

### first look

In [5]:
data.head(20)

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:40:00,NEW HANOVER,REINDEER CT & DEAD END,1
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:40:00,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 17:40:00,NORRISTOWN,HAWS AVE,1
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 17:40:01,NORRISTOWN,AIRY ST & SWEDE ST,1
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 17:40:01,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1
5,40.253473,-75.283245,CANNON AVE & W 9TH ST; LANSDALE; Station 345;...,19446.0,EMS: HEAD INJURY,2015-12-10 17:40:01,LANSDALE,CANNON AVE & W 9TH ST,1
6,40.182111,-75.127795,LAUREL AVE & OAKDALE AVE; HORSHAM; Station 35...,19044.0,EMS: NAUSEA/VOMITING,2015-12-10 17:40:01,HORSHAM,LAUREL AVE & OAKDALE AVE,1
7,40.217286,-75.405182,COLLEGEVILLE RD & LYWISKI RD; SKIPPACK; Stati...,19426.0,EMS: RESPIRATORY EMERGENCY,2015-12-10 17:40:01,SKIPPACK,COLLEGEVILLE RD & LYWISKI RD,1
8,40.289027,-75.399590,MAIN ST & OLD SUMNEYTOWN PIKE; LOWER SALFORD;...,19438.0,EMS: SYNCOPAL EPISODE,2015-12-10 17:40:01,LOWER SALFORD,MAIN ST & OLD SUMNEYTOWN PIKE,1
9,40.102398,-75.291458,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD; PLYM...,19462.0,Traffic: VEHICLE ACCIDENT -,2015-12-10 17:40:01,PLYMOUTH,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD,1


> # CLEANING

### Drop column 'e', which is the same for everything
### Change column names

In [6]:
data.columns = [each.replace('-','_').lower() for each in data.columns]
data = data.drop('e', 1)
data = data.rename(columns = {'twp' : 'town', 'addr' : 'address'})

data.head()

,lat,lng,desc,zip,title,timestamp,town,address
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:40:00,NEW HANOVER,REINDEER CT & DEAD END
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:40:00,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 17:40:00,NORRISTOWN,HAWS AVE
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 17:40:01,NORRISTOWN,AIRY ST & SWEDE ST
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 17:40:01,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END


### DESC is delimited with ';'
- 1st == addr *<-- drop*
- 2nd == town *<-- drop*
- 3rd == station *<-- create*
- 4th == timestamp
#### extract the station info

In [12]:
data['delimited'] = data['desc'].apply(lambda x:x.split(';')[0])

In [13]:
data.delimited

0                               REINDEER CT & DEAD END
1                           BRIAR PATH & WHITEMARSH LN
2                                             HAWS AVE
3                                   AIRY ST & SWEDE ST
4                             CHERRYWOOD CT & DEAD END
5                                CANNON AVE & W 9TH ST
6                             LAUREL AVE & OAKDALE AVE
7                         COLLEGEVILLE RD & LYWISKI RD
8                        MAIN ST & OLD SUMNEYTOWN PIKE
9             BLUEROUTE  & RAMP I476 NB TO CHEMICAL RD
10                               RT202 PKWY & KNAPP RD
11                               BROOK RD & COLWELL LN
12                       BYBERRY AVE & S WARMINSTER RD
13                             OLD YORK RD & VALLEY RD
14               SCHUYLKILL EXPY & CROTON RD UNDERPASS
15                             STUMP RD & WITCHWOOD DR
16                         SUSQUEHANNA AVE & W MAIN ST
17                             CHARLOTTE ST & MILES RD
18        

### Split 'title' into 'agency' and 'title'

### Fix Time & Date to expand all and assign day of week

In [25]:
data.date = data.timeStamp.str.extract('(....-..-..)', expand = True)
data.date

,0
0,2015-12-10
1,2015-12-10
2,2015-12-10
3,2015-12-10
4,2015-12-10
5,2015-12-10
6,2015-12-10
7,2015-12-10
8,2015-12-10
9,2015-12-10


In [10]:
data.time = data.timeStamp.str.extract('(..:..:..)', expand = True)
data.time

,0
0,17:40:00
1,17:40:00
2,17:40:00
3,17:40:01
4,17:40:01
5,17:40:01
6,17:40:01
7,17:40:01
8,17:40:01
9,17:40:01


### See about getting Zip from lat & lng; it might be easiest to just replace zip column to make it standard

### Split 'address' by the '&' into street_1 and street_2; if street_2 == null, set as 'none'

# NOTES
- maybe can use lat/lng to determine radiuses & time between calls in that area


## Ideas
- some way to map these once categorized?
- any way to animate it over time?
- are there any trends for the types of calls? ems / fire / types of chief complaint?


### CLASSIFY based on 
- ems / fire, etc
- town name ('twp')
- further split the calls into chief complaints
- time of day, day of week, month, etc


### GOAL :: Recommend geographical & time staffing for each type of emergency (fire, ems, trauma, ALS) 